In [1]:
from ipywidgets import FloatSlider, ValueWidget
import ipywidgets as widgets

from sidepanel import SidePanel

from regulus.utils import io
from regulus.topo import * 
from regulus.alg import *
from regulus.measures import *
from regulus.models import *
from regulus.tree import *

from ipyregulus import TreeWidget, TreeView, DetailsView, DataWidget
from ipyregulus.filters import * 

In [2]:
data = io.load('data/gauss4.p')
# data = io.load('data/simulations.p')

In [5]:
data.add_attr('linear', node_model(linear_model))
data.add_attr('fitness', fitness)
data.add_attr('relative_fitness', relative_fitness)

data.tree.add_attr('parent_fitness', parent_fitness)
data.tree.add_attr('child_fitness', child_fitness)
data.tree.add_attr('size', node_size)
data.tree.add_attr('rel_size', node_relative_size)
data.tree.add_attr('span', node_span)

In [6]:
from ipywidgets import Output
debug = Output(title='debug')
debug

Output()

### Helper functions

#### view

In [12]:
def update_view(view, f):
#     @debug.capture()
    def _inner():
        print('update_view')
        view.show = filter_tree(view.tree, f)
    return _inner 

In [13]:
def view(data, attr='span', func=lambda x, v: v <= x, title=''):
    v = TreeView(data, attr=attr)
    f = AttrFilter(attr=attr, func=func)
    m = Monitor(f, func=update_view(v, f))
    p = SidePanel(title=title)
    with p:
        display(v,f) 
    return v, f, p

#### reduce

In [14]:
def reduce_view(widget, tree, f):
    @debug.capture()
    def _inner():
        widget.ref = reduce_tree(tree, filter=f)
    return _inner

In [15]:
def reduce(data, attr='span', func=lambda x, v: v <= x, title=''):
    v = TreeView(data.tree, attr=attr)
    f = AttrFilter(attr=attr, func=func)
    m = Monitor(f, func=reduce_view(v, data.tree, f))
    p = SidePanel(title=title)
    with p:
        display(v,f) 
    return v, f, p

### Initial tree

In [25]:
tw = TreeWidget(data.tree)

In [26]:
v1, f1, p1 = view(tw, title='Initial Tree')

In [28]:
v2, f2, p2 = view(tw, attr='fitness', title='fitness')

### Reduce tree

In [29]:
v2, r2, p2 = reduce(tw, attr='span', title='reduce')

In [30]:
v2.attr = 'parent_fitness'

### fitness

In [39]:
v2

TreeView(field='parent_fitness', tree_model=TreeWidget(attrs={'parent_fitness': {0: 0, 4: 0.1458035673077832, …

#### test

In [31]:
tw2 = TreeWidget(data.tree)

In [34]:
f = AttrFilter(attr='span', func=lambda x, v: v <= x)

In [35]:
@debug.capture()
def test():
    print('test')
    tw2.ref = reduce_tree(tw.tree, filter=f)

In [36]:
m = Monitor(f, func=test)

In [37]:
test_view = TreeView(tw2, attr='span') 

In [38]:
p = SidePanel(title='Test')
with p:
    display(test_view, f)

In [32]:
fitness = view(tw2, attr='fitness', title='fitness')

In [33]:
parent_fitness = view(tw2, attr='parent_fitness', title='parent_fitness')

In [17]:
f_view = TreeView(tw2, attr='fitness')

HasTree ref assignment
HasTree observe owner TreeWidget(root=<regulus.tree.tree.Node object at 0x1a22ab81d0>)
HasTree ref_changed
TreeView update
TreeView update.  tree: <regulus.topo.regulus.RegulusTree object at 0x1a22ab8f60> owner: TreeWidget(root=<regulus.tree.tree.Node object at 0x1a22ab81d0>) attr: fitness


In [18]:
f_view

TreeView(field='fitness', tree_model=TreeWidget(attrs={'fitness': {0: 0.37583897392980714, 1: 0.38720959681741…

In [40]:
test_view

TreeView(field='span', tree_model=TreeWidget(attrs={'fitness': {0: 0.37583897392980714, 1: 0.3872095968174194,…

In [42]:
v1

TreeView(field='span', show={0, 4, 7, 14, 146, 147, 20, 21, 34, 166, 45, 48, 179, 180, 188, 64, 212, 215, 216,…